In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34305")
client

<Client: 'tcp://127.0.0.1:34305' processes=8 threads=40, memory=157.36 GiB>

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import os.path as op
import os
import gsw
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
chaos = '/tank/chaocean/tuchida/'
# savedir = '/tank/chaocean/tuchida/ECycle/'
savedir = '/tank/spectre/tuchida/chaocean/Snap/'
tdgs = '/tank/tdgs/tuchida/TWA/'
topog = '/tank/topog/tuchida/TWA/'

In [4]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

xchunk = 100
ychunk = 100

nmembs = np.arange(100,148)
ymax = 43
ymin = 14
xmin = 270
xmax = 337

In [5]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1576800,dnf).reshape(len(years),nfile)
dd = 70  # 
yystart = 0
ntimes[yystart,dd:]

array([1570320, 1572480, 1574640])

In [6]:
dsg = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                              % (100)), 
                      grid_dir=gdir,
                      prefix=['snap_diagOcn'], delta_t=2e2,
                      iters=ntimes[0,0], 
                     ).chunk({'YC':ychunk,'YG':ychunk,
                              'XC':xchunk,'XG':xchunk})
grid = Grid(dsg, periodic=['X'])
Vol = (dsg.rA * dsg.drF * dsg.hFacC).where(dsg.maskC!=0.).reset_coords(drop=True)
grid

<xgcm.Grid>
Y Axis (not periodic, boundary=None):
  * center   YC --> left
  * left     YG --> center
Z Axis (not periodic, boundary=None):
  * center   Z --> left
  * outer    Zp1 --> center
  * right    Zu --> center
  * left     Zl --> center
X Axis (periodic, boundary=None):
  * center   XC --> left
  * left     XG --> center
T Axis (not periodic, boundary=None):
  * center   time

In [8]:
from datetime import timedelta
time = np.arange('1967-01-01', '1968-01-01', timedelta(days=5), dtype='datetime64[D]')
time

array(['1967-01-01', '1967-01-06', '1967-01-11', '1967-01-16',
       '1967-01-21', '1967-01-26', '1967-01-31', '1967-02-05',
       '1967-02-10', '1967-02-15', '1967-02-20', '1967-02-25',
       '1967-03-02', '1967-03-07', '1967-03-12', '1967-03-17',
       '1967-03-22', '1967-03-27', '1967-04-01', '1967-04-06',
       '1967-04-11', '1967-04-16', '1967-04-21', '1967-04-26',
       '1967-05-01', '1967-05-06', '1967-05-11', '1967-05-16',
       '1967-05-21', '1967-05-26', '1967-05-31', '1967-06-05',
       '1967-06-10', '1967-06-15', '1967-06-20', '1967-06-25',
       '1967-06-30', '1967-07-05', '1967-07-10', '1967-07-15',
       '1967-07-20', '1967-07-25', '1967-07-30', '1967-08-04',
       '1967-08-09', '1967-08-14', '1967-08-19', '1967-08-24',
       '1967-08-29', '1967-09-03', '1967-09-08', '1967-09-13',
       '1967-09-18', '1967-09-23', '1967-09-28', '1967-10-03',
       '1967-10-08', '1967-10-13', '1967-10-18', '1967-10-23',
       '1967-10-28', '1967-11-02', '1967-11-07', '1967-

In [9]:
nmemb = 100
for itime in ntimes[yystart,:]:
    
    if itime == ntimes[yystart,0]:
        ds0 = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                          % (nmemb)), 
                                  grid_dir=gdir,
                                  prefix=['snap_diagOcn'], delta_t=2e2,
                                  iters=itime, 
                                 ).chunk({'YC':ychunk,'YG':ychunk,
                                                'XC':xchunk,'XG':xchunk})
    else:
        ds0 = xr.concat([ds0, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                          % (nmemb)), 
                                                  grid_dir=gdir,
                                                  prefix=['snap_diagOcn'], delta_t=2e2,
                                                  iters=itime, 
                                                 ).chunk({'YC':ychunk,'YG':ychunk,
                                                                'XC':xchunk,'XG':xchunk})
                           ], dim='time')

dsm = ds0.mean('time')
grid = Grid(dsm, periodic=['X'])

ub = dsm.UVEL
vb = dsm.VVEL
up = ds0.UVEL - dsm.UVEL
vp = ds0.VVEL - dsm.VVEL
    
MKE = .5*(grid.interp(ub**2,'X')
              + grid.interp(vb**2,'Y',boundary='extend')
             ).where(dsg.maskC!=0.).reset_coords(drop=True)
EKE = .5*(grid.interp(up**2,'X')
              + grid.interp(vp**2,'Y',boundary='extend')
             ).where(dsg.maskC!=0.).reset_coords(drop=True).mean('time')
    
EKE

<xarray.DataArray (Z: 46, YC: 900, XC: 1000)>
dask.array<mul, shape=(46, 900, 1000), dtype=float32, chunksize=(46, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * Z        (Z) >f4 -3.047 -9.454 -16.36 ... -5.375e+03 -5.625e+03 -5.875e+03

In [10]:
dsave = EKE.to_dataset(name='EKE')
dsave['MKE'] = MKE
dsave.to_zarr(op.join(savedir,'%4d/Time-decomp/KEs.zarr' 
                      % (years[yystart])), mode='w')
print(dsave)
dsave.close()

<xarray.Dataset>
Dimensions:  (XC: 1000, YC: 900, Z: 46)
Coordinates:
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * Z        (Z) >f4 -3.047 -9.454 -16.36 ... -5.375e+03 -5.625e+03 -5.875e+03
Data variables:
    EKE      (Z, YC, XC) float32 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    MKE      (Z, YC, XC) float32 dask.array<chunksize=(46, 100, 100), meta=np.ndarray>


In [8]:
EKE = xr.open_zarr(op.join(savedir,'%4d/Time-decomp/KEs.zarr' 
                                   % (years[yystart]))
                          ).EKE
MKE = xr.open_zarr(op.join(savedir,'%4d/Time-decomp/KEs.zarr' 
                                   % (years[yystart]))
                          ).MKE

In [13]:
ymax = 43
ymin = 14
xmin = 270
xmax = 337
((EKE.where(dsg.maskC!=0.)*Vol).sel(
            Z=slice(None,-1e3),YC=slice(ymin,ymax),
            XC=slice(xmin,xmax)
            ).sum(['Z','YC','XC'],skipna=True)
 / Vol.sel(
           Z=slice(None,-1e3),YC=slice(ymin,ymax),
           XC=slice(xmin,xmax)
          ).sum(['Z','YC','XC'],skipna=True)
).compute()

<xarray.DataArray ()>
array(0.0094047, dtype=float32)

In [14]:
((MKE.where(dsg.maskC!=0.)*Vol).sel(
            Z=slice(None,-1e3),YC=slice(ymin,ymax),
            XC=slice(xmin,xmax)
            ).sum(['Z','YC','XC'],skipna=True)
 / Vol.sel(
           Z=slice(None,-1e3),YC=slice(ymin,ymax),
           XC=slice(xmin,xmax)
          ).sum(['Z','YC','XC'],skipna=True)
).compute()

<xarray.DataArray ()>
array(0.00596205, dtype=float32)